In [1]:
import os
import cv2
import glob
import numpy as np
import time

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'background subtraction (SHOWCASE).ipynb',
 'background subtraction slow classification.ipynb',
 'background subtraction.ipynb',
 'Dense optical flow cuda.ipynb',
 'Dense optical flow.ipynb',
 'frame difference-Copy1.ipynb',
 'frame difference.ipynb',
 'FRAME_BACKGROUND_DIFFERENCE (SHOWCASE).ipynb',
 'images',
 'learning.ipynb',
 'lucas kanade optical flow.ipynb',
 'moc (SHOWCASE)-Copy2.py',
 'MOC Alarm (SHOWCASE).py',
 'moc(SHOWCASE)-Copy1.py',
 'motion object classifier (SHOWCASE).ipynb',
 'motion_detection_0.ipynb',
 'motion_detection_geeks for feeks.ipynb',
 'motion_detection_learning.ipynb',
 'mp learning.py',
 'non max supression.ipynb',
 'object detection (SHOWCASE).ipynb',
 'opencv_cuda (SHOWCASE).ipynb',
 'opencv_cuda-Copy1.ipynb',
 'opencv_cuda.ipynb',
 'optical flow 2.ipynb',
 'optical flow.ipynb',
 'other videos',
 'pt models',
 'sounds',
 'turkish_labels.pickle',
 'turkish_labels_2.pickle',
 'video_footages',
 'ymotion test.ipynb',
 'yolov8l.pt',
 '

In [3]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\snow dispute.mp4',
 'video_footages\\Surveillance camera captures tanks entering Ukraine via Belarus border.mp4',
 'video_footages\\trail cam video creatures of the night.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4',
 'video_footages\\차흐르듯 (1).mp4',
 'video_footages\\차흐르듯.mp4']

In [4]:
def get_motion_mask(flow_mag, motion_thresh=1, kernel=np.ones((7,7))):

    motion_mask = np.uint8(flow_mag > motion_thresh)*255

    motion_mask = cv2.erode(motion_mask, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    
    return motion_mask

In [5]:
# init dict to track time for every stage at each iteration
timers = {
    "full pipeline": [],
    "reading": [],
    "pre-process": [],
    "post-process": [],
    "optical flow": [],
}

mask_kernel = np.ones((7,7), dtype=np.uint8)

thresh = 250


# init video capture with video
for video in video_files:
    cap = cv2.VideoCapture(video)
    
    # get default video FPS
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # get total number of video frames
    num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
        # read the first frame
    ret, previous_frame = cap.read()
    h, w, l = previous_frame.shape
    n_h = 360 #int(h/2)
    n_w = 600 #int(w/2)
    
    motion_thresh = np.c_[np.linspace(0.3, 1, n_h)].repeat(n_w, axis=-1)
    
    # proceed if frame reading was successful
    if ret:
        # resize frame
        frame = cv2.resize(previous_frame, (n_w, n_h))
    
        # upload resized frame to GPU
        gpu_frame = cv2.cuda_GpuMat()
        gpu_frame.upload(frame)
    
        # convert to gray
        previous_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # upload pre-processed frame to GPU
        gpu_previous = cv2.cuda_GpuMat()
        gpu_previous.upload(previous_frame)
    
        # create gpu_hsv output for optical flow
        gpu_hsv = cv2.cuda_GpuMat(gpu_frame.size(), cv2.CV_32FC3)
        gpu_hsv_8u = cv2.cuda_GpuMat(gpu_frame.size(), cv2.CV_8UC3)
    
        gpu_h = cv2.cuda_GpuMat(gpu_frame.size(), cv2.CV_32FC1)
        gpu_s = cv2.cuda_GpuMat(gpu_frame.size(), cv2.CV_32FC1)
        gpu_v = cv2.cuda_GpuMat(gpu_frame.size(), cv2.CV_32FC1)
    
        # set saturation to 1
        gpu_s.upload(np.ones_like(previous_frame, np.float32))
    
        while True:
            # start full pipeline timer
            start_full_time = time.time()
    
            # start reading timer
            start_read_time = time.time()
    
            # capture frame-by-frame
            ret, frame = cap.read()
    
            # upload frame to GPU
            gpu_frame.upload(frame)
    
            # end reading timer
            end_read_time = time.time()
    
            # add elapsed iteration time
            timers["reading"].append(end_read_time - start_read_time)
    
            # if frame reading was not successful, break
            if not ret:
                break
    
            # start pre-process timer
            start_pre_time = time.time()
    
            # resize frame
            gpu_frame = cv2.cuda.resize(gpu_frame, (n_w, n_h))
    
            # convert to gray
            gpu_current = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2GRAY)
    
            # end pre-process timer
            end_pre_time = time.time()
    
            # add elapsed iteration time
            timers["pre-process"].append(end_pre_time - start_pre_time)
    
            # start optical flow timer
            start_of = time.time()
    
            # create optical flow instance
            gpu_flow = cv2.cuda_FarnebackOpticalFlow.create(
                    5, 0.5, False, 15, 3, 5, 1.2, 0,
            )
            # calculate optical flow
            gpu_flow = cv2.cuda_FarnebackOpticalFlow.calc(
                gpu_flow, gpu_previous, gpu_current, None,
            )
    
            # end of timer
            end_of = time.time()
    
            # add elapsed iteration time
            timers["optical flow"].append(end_of - start_of)
    
            # start post-process timer
            start_post_time = time.time()
    
            gpu_flow_x = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
            gpu_flow_y = cv2.cuda_GpuMat(gpu_flow.size(), cv2.CV_32FC1)
            cv2.cuda.split(gpu_flow, [gpu_flow_x, gpu_flow_y])
    
            # convert from cartesian to polar coordinates to get magnitude and angle
            gpu_magnitude, gpu_angle = cv2.cuda.cartToPolar(
                gpu_flow_x, gpu_flow_y, angleInDegrees=True,
            )
    
            # set value to normalized magnitude from 0 to 1
            gpu_v = cv2.cuda.normalize(gpu_magnitude, 0.0, 1.0, cv2.NORM_MINMAX, -1)
    
            # get angle of optical flow
            angle = gpu_angle.download()
            angle *= (1 / 360.0) * (180 / 255.0)
    
            # set hue according to the angle of optical flow
            gpu_h.upload(angle)
    
            # merge h,s,v channels
            cv2.cuda.merge([gpu_h, gpu_s, gpu_v], gpu_hsv)
    
            # multiply each pixel value to 255
            gpu_hsv.convertTo(cv2.CV_8U, gpu_hsv_8u, 255)
    
            # convert hsv to bgr
            gpu_bgr = cv2.cuda.cvtColor(gpu_hsv_8u, cv2.COLOR_HSV2BGR)
    
            # send original frame from GPU back to CPU
            frame = gpu_frame.download()
    
            # send result from GPU back to CPU
            bgr = gpu_bgr.download()
    
            mag = gpu_magnitude.download()
    
            motion_mask = get_motion_mask(mag, motion_thresh=motion_thresh, kernel=mask_kernel)
    
            contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
    
            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)
                area = w*h
                if area > thresh:
                    frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), thickness = 2)
    
            # update previous_frame value
            gpu_previous = gpu_current
    
            # end post-process timer
            end_post_time = time.time()
    
            # add elapsed iteration time
            timers["post-process"].append(end_post_time - start_post_time)
    
            # end full pipeline timer
            end_full_time = time.time()
    
            # add elapsed iteration time
            timers["full pipeline"].append(end_full_time - start_full_time)
    
            # visualization
            cv2.imshow("original", frame)
            #cv2.imshow("result", bgr)
            #cv2.imshow("mask", motion_mask)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    # release the capture
    cap.release()
    
    # destroy all windows
    cv2.destroyAllWindows()
    
    # print results
    print("Number of frames : ", num_frames)
    
    # elapsed time at each stage
    print("Elapsed time")
    for stage, seconds in timers.items():
        print("-", stage, ": {:0.3f} seconds".format(sum(seconds)))
    
        # calculate frames per second
        print("Default video FPS : {:0.3f}".format(fps))
    
        of_fps = (num_frames - 1) / sum(timers["optical flow"])
        print("Optical flow FPS : {:0.3f}".format(of_fps))
    
        full_fps = (num_frames - 1) / sum(timers["full pipeline"])
        print("Full pipeline FPS : {:0.3f}".format(full_fps))

Number of frames :  1654.0
Elapsed time
- full pipeline : 13.818 seconds
Default video FPS : 29.970
Optical flow FPS : 204.246
Full pipeline FPS : 119.625
- reading : 2.094 seconds
Default video FPS : 29.970
Optical flow FPS : 204.246
Full pipeline FPS : 119.625
- pre-process : 0.473 seconds
Default video FPS : 29.970
Optical flow FPS : 204.246
Full pipeline FPS : 119.625
- post-process : 3.147 seconds
Default video FPS : 29.970
Optical flow FPS : 204.246
Full pipeline FPS : 119.625
- optical flow : 8.093 seconds
Default video FPS : 29.970
Optical flow FPS : 204.246
Full pipeline FPS : 119.625
Number of frames :  1343.0
Elapsed time
- full pipeline : 33.471 seconds
Default video FPS : 23.976
Optical flow FPS : 67.812
Full pipeline FPS : 40.095
- reading : 4.997 seconds
Default video FPS : 23.976
Optical flow FPS : 67.812
Full pipeline FPS : 40.095
- pre-process : 1.155 seconds
Default video FPS : 23.976
Optical flow FPS : 67.812
Full pipeline FPS : 40.095
- post-process : 7.500 seconds